# Model Prediction Analysis - Feedforward Neural Network

This notebook provides comprehensive analysis and visualization of the trained FFNN predictions on the Parkinson's telemonitoring dataset.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# Load predictions
predictions_df = pd.read_csv('../data/predictions/predictions.csv')

print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"\nTotal predictions: {len(predictions_df)}")
print(f"\nColumns: {list(predictions_df.columns)}")
print(f"\nDataset shape: {predictions_df.shape}")
print("\nFirst few predictions:")
print(predictions_df[['motor_UPDRS', 'pred_motor_UPDRS', 'total_UPDRS', 'pred_total_UPDRS']].head(10))

## 1. Overall Performance Metrics

In [ ]:
# Calculate metrics for both targets
def calculate_metrics(y_true, y_pred, name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # MAPE
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    
    print(f"\n{'=' * 60}")
    print(f"{name:^60}")
    print(f"{'=' * 60}")
    print(f"MSE (Mean Squared Error):        {mse:10.4f}")
    print(f"RMSE (Root Mean Squared Error):  {rmse:10.4f}")
    print(f"MAE (Mean Absolute Error):       {mae:10.4f}")
    print(f"R² Score:                        {r2:10.4f}")
    print(f"MAPE (Mean Absolute % Error):    {mape:9.2f}%")
    print(f"{'=' * 60}")
    
    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'MAPE': mape}

# Motor UPDRS
motor_metrics = calculate_metrics(
    predictions_df['motor_UPDRS'],
    predictions_df['pred_motor_UPDRS'],
    'Motor UPDRS Metrics'
)

# Total UPDRS
total_metrics = calculate_metrics(
    predictions_df['total_UPDRS'],
    predictions_df['pred_total_UPDRS'],
    'Total UPDRS Metrics'
)

## 2. Prediction vs Actual Scatter Plots

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Motor UPDRS
axes[0].scatter(predictions_df['motor_UPDRS'], predictions_df['pred_motor_UPDRS'], 
                alpha=0.5, s=20, color='steelblue')
axes[0].plot([predictions_df['motor_UPDRS'].min(), predictions_df['motor_UPDRS'].max()],
             [predictions_df['motor_UPDRS'].min(), predictions_df['motor_UPDRS'].max()],
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Motor UPDRS', fontsize=12)
axes[0].set_ylabel('Predicted Motor UPDRS', fontsize=12)
axes[0].set_title(f'Motor UPDRS: Predicted vs Actual\nR² = {motor_metrics["R2"]:.4f}', 
                  fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Total UPDRS
axes[1].scatter(predictions_df['total_UPDRS'], predictions_df['pred_total_UPDRS'], 
                alpha=0.5, s=20, color='coral')
axes[1].plot([predictions_df['total_UPDRS'].min(), predictions_df['total_UPDRS'].max()],
             [predictions_df['total_UPDRS'].min(), predictions_df['total_UPDRS'].max()],
             'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Total UPDRS', fontsize=12)
axes[1].set_ylabel('Predicted Total UPDRS', fontsize=12)
axes[1].set_title(f'Total UPDRS: Predicted vs Actual\nR² = {total_metrics["R2"]:.4f}', 
                  fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Residual Analysis

In [ ]:
# Calculate residuals
motor_residuals = predictions_df['motor_UPDRS'] - predictions_df['pred_motor_UPDRS']
total_residuals = predictions_df['total_UPDRS'] - predictions_df['pred_total_UPDRS']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Motor UPDRS - Residual plot
axes[0, 0].scatter(predictions_df['pred_motor_UPDRS'], motor_residuals, 
                   alpha=0.5, s=20, color='steelblue')
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Predicted Motor UPDRS')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Motor UPDRS: Residual Plot', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Motor UPDRS - Residual distribution
axes[0, 1].hist(motor_residuals, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 1].axvline(motor_residuals.mean(), color='red', linestyle='--', 
                   label=f'Mean: {motor_residuals.mean():.2f}')
axes[0, 1].set_xlabel('Residuals')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Motor UPDRS: Residual Distribution', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Total UPDRS - Residual plot
axes[1, 0].scatter(predictions_df['pred_total_UPDRS'], total_residuals, 
                   alpha=0.5, s=20, color='coral')
axes[1, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Predicted Total UPDRS')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].set_title('Total UPDRS: Residual Plot', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Total UPDRS - Residual distribution
axes[1, 1].hist(total_residuals, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1, 1].axvline(total_residuals.mean(), color='red', linestyle='--',
                   label=f'Mean: {total_residuals.mean():.2f}')
axes[1, 1].set_xlabel('Residuals')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Total UPDRS: Residual Distribution', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("RESIDUAL STATISTICS")
print("=" * 60)
print(f"\nMotor UPDRS Residuals:")
print(f"  Mean:     {motor_residuals.mean():8.4f}")
print(f"  Std:      {motor_residuals.std():8.4f}")
print(f"  Skewness: {motor_residuals.skew():8.4f}")
print(f"  Kurtosis: {motor_residuals.kurtosis():8.4f}")

print(f"\nTotal UPDRS Residuals:")
print(f"  Mean:     {total_residuals.mean():8.4f}")
print(f"  Std:      {total_residuals.std():8.4f}")
print(f"  Skewness: {total_residuals.skew():8.4f}")
print(f"  Kurtosis: {total_residuals.kurtosis():8.4f}")

## 4. Error Distribution by Prediction Range

In [ ]:
# Bin predictions and analyze errors
predictions_df['motor_bin'] = pd.cut(predictions_df['motor_UPDRS'], bins=5)
predictions_df['total_bin'] = pd.cut(predictions_df['total_UPDRS'], bins=5)

predictions_df['motor_abs_error'] = np.abs(motor_residuals)
predictions_df['total_abs_error'] = np.abs(total_residuals)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Motor UPDRS error by range
motor_grouped = predictions_df.groupby('motor_bin')['motor_abs_error'].mean()
motor_grouped.plot(kind='bar', ax=axes[0], color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Motor UPDRS Range')
axes[0].set_ylabel('Mean Absolute Error')
axes[0].set_title('Motor UPDRS: MAE by Actual Value Range', fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3, axis='y')

# Total UPDRS error by range
total_grouped = predictions_df.groupby('total_bin')['total_abs_error'].mean()
total_grouped.plot(kind='bar', ax=axes[1], color='coral', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Total UPDRS Range')
axes[1].set_ylabel('Mean Absolute Error')
axes[1].set_title('Total UPDRS: MAE by Actual Value Range', fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Q-Q Plots for Residuals (Normality Check)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Motor UPDRS Q-Q plot
stats.probplot(motor_residuals, dist="norm", plot=axes[0])
axes[0].set_title('Motor UPDRS: Residuals Q-Q Plot', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Total UPDRS Q-Q plot
stats.probplot(total_residuals, dist="norm", plot=axes[1])
axes[1].set_title('Total UPDRS: Residuals Q-Q Plot', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Shapiro-Wilk test for normality
motor_shapiro = stats.shapiro(motor_residuals.sample(min(5000, len(motor_residuals))))
total_shapiro = stats.shapiro(total_residuals.sample(min(5000, len(total_residuals))))

print("\n" + "=" * 60)
print("NORMALITY TESTS (Shapiro-Wilk)")
print("=" * 60)
print(f"\nMotor UPDRS Residuals:")
print(f"  Statistic: {motor_shapiro.statistic:.6f}")
print(f"  p-value:   {motor_shapiro.pvalue:.6f}")
print(f"  Normal?    {'Yes' if motor_shapiro.pvalue > 0.05 else 'No (p < 0.05)'}")

print(f"\nTotal UPDRS Residuals:")
print(f"  Statistic: {total_shapiro.statistic:.6f}")
print(f"  p-value:   {total_shapiro.pvalue:.6f}")
print(f"  Normal?    {'Yes' if total_shapiro.pvalue > 0.05 else 'No (p < 0.05)'}")

## 6. Prediction Accuracy by Subject

In [ ]:
# Analyze per-subject performance
subject_errors = predictions_df.groupby('subject#').agg({
    'motor_abs_error': 'mean',
    'total_abs_error': 'mean'
}).reset_index()

subject_errors.columns = ['subject#', 'motor_MAE', 'total_MAE']

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Motor UPDRS per subject
axes[0].hist(subject_errors['motor_MAE'], bins=30, edgecolor='black', 
             alpha=0.7, color='steelblue')
axes[0].axvline(subject_errors['motor_MAE'].mean(), color='red', 
                linestyle='--', linewidth=2, label=f'Mean: {subject_errors["motor_MAE"].mean():.2f}')
axes[0].set_xlabel('Mean Absolute Error per Subject')
axes[0].set_ylabel('Number of Subjects')
axes[0].set_title('Motor UPDRS: MAE Distribution Across Subjects', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Total UPDRS per subject
axes[1].hist(subject_errors['total_MAE'], bins=30, edgecolor='black', 
             alpha=0.7, color='coral')
axes[1].axvline(subject_errors['total_MAE'].mean(), color='red',
                linestyle='--', linewidth=2, label=f'Mean: {subject_errors["total_MAE"].mean():.2f}')
axes[1].set_xlabel('Mean Absolute Error per Subject')
axes[1].set_ylabel('Number of Subjects')
axes[1].set_title('Total UPDRS: MAE Distribution Across Subjects', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("PER-SUBJECT ERROR STATISTICS")
print("=" * 60)
print(f"\nNumber of subjects: {len(subject_errors)}")
print(f"\nMotor UPDRS MAE per subject:")
print(f"  Mean: {subject_errors['motor_MAE'].mean():.4f}")
print(f"  Std:  {subject_errors['motor_MAE'].std():.4f}")
print(f"  Min:  {subject_errors['motor_MAE'].min():.4f}")
print(f"  Max:  {subject_errors['motor_MAE'].max():.4f}")

print(f"\nTotal UPDRS MAE per subject:")
print(f"  Mean: {subject_errors['total_MAE'].mean():.4f}")
print(f"  Std:  {subject_errors['total_MAE'].std():.4f}")
print(f"  Min:  {subject_errors['total_MAE'].min():.4f}")
print(f"  Max:  {subject_errors['total_MAE'].max():.4f}")

# Find best and worst performing subjects
print(f"\n\nBest performing subjects (Motor UPDRS):")
print(subject_errors.nsmallest(5, 'motor_MAE')[['subject#', 'motor_MAE']])

print(f"\nWorst performing subjects (Motor UPDRS):")
print(subject_errors.nlargest(5, 'motor_MAE')[['subject#', 'motor_MAE']])

## 7. Temporal Prediction Analysis (Sample Subjects)

In [ ]:
# Select subjects with most data points
subject_counts = predictions_df['subject#'].value_counts()
top_subjects = subject_counts.head(6).index

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, subject_id in enumerate(top_subjects):
    subject_data = predictions_df[predictions_df['subject#'] == subject_id].sort_values('test_time')
    
    axes[idx].plot(subject_data['test_time'], subject_data['motor_UPDRS'], 
                   marker='o', linewidth=2, label='Actual Motor', color='steelblue')
    axes[idx].plot(subject_data['test_time'], subject_data['pred_motor_UPDRS'], 
                   marker='s', linewidth=2, label='Predicted Motor', 
                   color='lightblue', linestyle='--')
    axes[idx].plot(subject_data['test_time'], subject_data['total_UPDRS'], 
                   marker='o', linewidth=2, label='Actual Total', color='coral')
    axes[idx].plot(subject_data['test_time'], subject_data['pred_total_UPDRS'], 
                   marker='s', linewidth=2, label='Predicted Total', 
                   color='lightsalmon', linestyle='--')
    
    axes[idx].set_xlabel('Test Time (days)', fontsize=9)
    axes[idx].set_ylabel('UPDRS Score', fontsize=9)
    axes[idx].set_title(f'Subject {subject_id} ({len(subject_data)} recordings)', 
                       fontsize=10, fontweight='bold')
    axes[idx].legend(fontsize=7, loc='best')
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Actual vs Predicted UPDRS Over Time - Sample Subjects', 
             fontsize=14, fontweight='bold', y=1.001)
plt.tight_layout()
plt.show()

## 8. Error Correlation Analysis

In [ ]:
# Analyze correlation between errors
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Absolute errors correlation
axes[0].scatter(predictions_df['motor_abs_error'], predictions_df['total_abs_error'],
                alpha=0.5, s=20, color='purple')
axes[0].set_xlabel('Motor UPDRS Absolute Error')
axes[0].set_ylabel('Total UPDRS Absolute Error')
axes[0].set_title('Correlation between Motor and Total UPDRS Errors', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Correlation coefficient
error_corr = np.corrcoef(predictions_df['motor_abs_error'], 
                         predictions_df['total_abs_error'])[0, 1]
axes[0].text(0.05, 0.95, f'Correlation: {error_corr:.3f}',
             transform=axes[0].transAxes, fontsize=11,
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Actual values correlation
axes[1].scatter(predictions_df['motor_UPDRS'], predictions_df['total_UPDRS'],
                alpha=0.3, s=20, label='Actual', color='steelblue')
axes[1].scatter(predictions_df['pred_motor_UPDRS'], predictions_df['pred_total_UPDRS'],
                alpha=0.3, s=20, label='Predicted', color='coral')
axes[1].set_xlabel('Motor UPDRS')
axes[1].set_ylabel('Total UPDRS')
axes[1].set_title('Motor vs Total UPDRS: Actual vs Predicted', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("ERROR CORRELATION ANALYSIS")
print("=" * 60)
print(f"\nCorrelation between Motor and Total UPDRS errors: {error_corr:.4f}")
print(f"\nThis suggests that errors in predicting Motor UPDRS are", end=' ')
if abs(error_corr) > 0.7:
    print("strongly correlated with errors in Total UPDRS.")
elif abs(error_corr) > 0.4:
    print("moderately correlated with errors in Total UPDRS.")
else:
    print("weakly correlated with errors in Total UPDRS.")

## 9. Feature Impact on Prediction Error

In [ ]:
# Analyze how demographic features affect prediction error
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Error by age
age_bins = pd.cut(predictions_df['age'], bins=5)
age_errors = predictions_df.groupby(age_bins)['motor_abs_error'].mean()
age_errors.plot(kind='bar', ax=axes[0, 0], color='steelblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Age Range')
axes[0, 0].set_ylabel('Mean Absolute Error (Motor UPDRS)')
axes[0, 0].set_title('Prediction Error by Age Range', fontweight='bold')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Error by gender
gender_errors = predictions_df.groupby('sex').agg({
    'motor_abs_error': 'mean',
    'total_abs_error': 'mean'
})
gender_errors.plot(kind='bar', ax=axes[0, 1], color=['steelblue', 'coral'], 
                   alpha=0.7, edgecolor='black')
axes[0, 1].set_xlabel('Gender (0=Male, 1=Female)')
axes[0, 1].set_ylabel('Mean Absolute Error')
axes[0, 1].set_title('Prediction Error by Gender', fontweight='bold')
axes[0, 1].tick_params(axis='x', rotation=0)
axes[0, 1].legend(['Motor UPDRS', 'Total UPDRS'])
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Error over time
time_bins = pd.cut(predictions_df['test_time'], bins=10)
time_errors = predictions_df.groupby(time_bins)['motor_abs_error'].mean()
time_errors.plot(kind='line', ax=axes[1, 0], marker='o', color='steelblue', linewidth=2)
axes[1, 0].set_xlabel('Test Time (days)')
axes[1, 0].set_ylabel('Mean Absolute Error (Motor UPDRS)')
axes[1, 0].set_title('Prediction Error Over Time', fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Best vs worst predictions
predictions_df['motor_error_percentile'] = pd.qcut(predictions_df['motor_abs_error'], 
                                                     q=4, labels=['Q1 (Best)', 'Q2', 'Q3', 'Q4 (Worst)'])
quartile_errors = predictions_df.groupby('motor_error_percentile').agg({
    'motor_abs_error': 'mean',
    'total_abs_error': 'mean'
})
quartile_errors.plot(kind='bar', ax=axes[1, 1], color=['steelblue', 'coral'],
                     alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Error Quartile')
axes[1, 1].set_ylabel('Mean Absolute Error')
axes[1, 1].set_title('Error Distribution by Quartile', fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].legend(['Motor UPDRS', 'Total UPDRS'])
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 10. Summary and Key Findings

In [ ]:
print("\n" + "=" * 80)
print("FEEDFORWARD NEURAL NETWORK - PREDICTION ANALYSIS SUMMARY")
print("=" * 80)

print("\n1. OVERALL PERFORMANCE:")
print(f"   - Motor UPDRS R²: {motor_metrics['R2']:.4f} ({motor_metrics['R2']*100:.1f}% variance explained)")
print(f"   - Total UPDRS R²: {total_metrics['R2']:.4f} ({total_metrics['R2']*100:.1f}% variance explained)")
print(f"   - Motor UPDRS RMSE: {motor_metrics['RMSE']:.2f}")
print(f"   - Total UPDRS RMSE: {total_metrics['RMSE']:.2f}")

print("\n2. RESIDUAL CHARACTERISTICS:")
print(f"   - Motor residuals mean: {motor_residuals.mean():.4f} (close to 0 is good)")
print(f"   - Total residuals mean: {total_residuals.mean():.4f} (close to 0 is good)")
print(f"   - Motor residuals std: {motor_residuals.std():.2f}")
print(f"   - Total residuals std: {total_residuals.std():.2f}")

print("\n3. PER-SUBJECT VARIABILITY:")
print(f"   - Number of subjects: {len(subject_errors)}")
print(f"   - Motor MAE range across subjects: {subject_errors['motor_MAE'].min():.2f} - {subject_errors['motor_MAE'].max():.2f}")
print(f"   - Total MAE range across subjects: {subject_errors['total_MAE'].min():.2f} - {subject_errors['total_MAE'].max():.2f}")

print("\n4. ERROR CORRELATION:")
print(f"   - Correlation between Motor and Total errors: {error_corr:.4f}")

print("\n5. MODEL INSIGHTS:")
if motor_metrics['R2'] > 0.6:
    print("   ✓ Good predictive performance on Motor UPDRS")
else:
    print("   ⚠ Moderate predictive performance on Motor UPDRS")
    
if total_metrics['R2'] > 0.6:
    print("   ✓ Good predictive performance on Total UPDRS")
else:
    print("   ⚠ Moderate predictive performance on Total UPDRS")
    
if abs(motor_residuals.mean()) < 0.5 and abs(total_residuals.mean()) < 0.5:
    print("   ✓ Residuals centered around zero (unbiased predictions)")
else:
    print("   ⚠ Some bias detected in predictions")

print("\n6. RECOMMENDATIONS:")
print("   - Consider ensemble methods to reduce variance")
print("   - Investigate high-error subjects for data quality issues")
print("   - Explore temporal features to capture disease progression")
print("   - Try regularization techniques if overfitting is observed")

print("\n" + "=" * 80)